<a href="https://colab.research.google.com/github/CBravoR/AdvancedAnalyticsLabs/blob/master/notebooks/python/Lab_PD_Calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PD Calibration

In this lab we will learn how to estimate the long-run PD after a model has been trained. The PD calibration can be done with the score, the monthly portfolio each case belongs to (usually the behavioural scorecard is used), the labels (Default / Non-Default) and a set of economic factors. For this work we will use an exchange rate and a commodity price.

First, we load the data. It is in Excel, so we use the appropriate function from Pandas. There are two worksheets: The first one contains the data for each borrower and each portfolio, and the second one contains the macro factor at each month.

In [ ]:
!pip install fastexcel
!pip install pwlf

In [ ]:
# Base and data reading
import pandas as pd
import polars as pl
import numpy as np
from itertools import product

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Transformation
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Curve cutting
import pwlf

# Bigger and prettier plots
plt.rcParams['figure.figsize'] = (10, 5)
plt.style.use('fivethirtyeight')

# Date and time
from datetime import date
from dateutil.relativedelta import *

# Time series
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Normal function from scipy
from scipy.stats import norm


In [ ]:
# Download Excel file
!gdown 'https://drive.google.com/uc?id=1UYmgsu5gI5U_VbraKXHxWTXyZSbM6q5S'

In [ ]:
# Load the data. Two datasets are necessary.
loans = pl.read_excel('PDCalExample.xlsx', # Filename
                      sheet_id=1,         # Worksheet name or index
                      )

econ_factors = pl.read_excel('PDCalExample.xlsx', # Filename
                             sheet_id=2,         # Worksheet name or index
                             )

In [ ]:
econ_factors.head()

In [ ]:
loans.head()

In [ ]:
f"There are {loans.select("Portfolio").unique().count().to_numpy().item()} unique monthly portfolios"

In [ ]:
loans.describe()

In [ ]:
econ_factors.describe()

Let's normalize the economic factors using a z-transform.

In [ ]:
# Define the transformer and set the output to Polars
transformer = ColumnTransformer(
    transformers=[
        ("scaler", StandardScaler(), ["Commodity", "ExchangeRate"])
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
)
transformer.set_output(transform="polars")

# Fit and transform the data
econ_factors = transformer.fit_transform(econ_factors)

# Check the output
econ_factors.head()




## Defining Ratings

We have all the data we need. Let's start then by obtaining PD segments. Basel suggests building 7-15 segments. For this, we can use the excellent package [pwlf](https://pypi.org/project/pwlf/). It will allow segmenting a curve using a given number of cuts.

Which curve do we need to segment? The ROC curve!

In [ ]:
# Calculate the ROC curve points
fpr, tpr, thresholds = roc_curve(loans['Default'],
                                 loans['Probs'])

# Save the AUC in a variable to display it. Round it first
auc = np.round(roc_auc_score(y_true = loans['Default'],
                             y_score = loans['Probs']),
               decimals = 3)

# Create and show the plot
plt.plot(fpr,tpr,label="Scorecard, auc="+str(auc))
plt.legend(loc=4)
plt.show()

Now we can segment the curve. The process takes a while to run, and sadly it is sequential, so go make yourself a coffee / tea while this runs.

In [ ]:
# Define the curve with the ROC curve
piecewise_AUC = pwlf.PiecewiseLinFit(fpr, tpr)

In [ ]:
# Calculate the best curve. Long!
res = piecewise_AUC.fit(10, disp=True, seed=20251127)

As this is a long process, it is a good idea to save the results. The object res can be pickled, or simply the cuts can be saved in a commented line.

In [ ]:
res

In [ ]:
# Use previous result
# res = [0.        , 0.01383389, 0.03920524, 0.07731607, 0.11285577,
#      0.20653665, 0.32339978, 0.41818781, 0.57283343, 0.7999917 ,
#      1.
#      ]

In [ ]:
ROC_curve = pd.DataFrame({'fpr': fpr, 'threshold': thresholds})
ROC_curve

To apply the cuts, you can use the method ```fit_with_breaks``` that is available for the ```piecewise_AUC``` object.


In [ ]:
# Apply cuts!
piecewise_AUC.fit_with_breaks(res)

We can now apply this to our dataset and see how the piecewise curve fits the ROC curve.

In [ ]:
# predict for the determined points
xHat = np.linspace(min(fpr), max(fpr), num=10000)
yHat = piecewise_AUC.predict(xHat)

# plot the results
plt.figure()
plt.plot(fpr, tpr, 'o')
plt.plot(xHat, yHat, '-')
plt.show()

We have a pretty good fit! But we are still not done, we now need to understand if the cuts lead to monotonic PDs. This is the final constraint. For this, we first calculate the PD for the whole dataset (we will adjust this later for each portfolio).

In [ ]:
res

In [ ]:
# Find probability associated with every cut
pbb_cuts = np.zeros_like(res)
i = 0

# Find the threshold value that correspond to the closest point in the ROC_curve object's fpr, to each element of the res object.
for element in res:
  pbb_cuts[i] = ROC_curve.loc[ROC_curve['fpr'].sub(element).abs().idxmin(), 'threshold']
  i += 1

# Reverse the vector
pbb_cuts = np.flip(pbb_cuts)

# Show the outcome
pbb_cuts

In [ ]:
# Insert 0 in the first position, and replace the last value to one.
pbb_cuts = np.insert(pbb_cuts, 0, 0)
pbb_cuts[-1] = 1
pbb_cuts

In [ ]:
# Segment the probabilities
pd_cut = pd.cut(loans['Probs'], pbb_cuts)
pd_cut

And we study the output with a crosstab.

In [ ]:
# Create table with cases total.
PDs_Tab = pd.crosstab(pd_cut,
                      loans['Default'],
                      normalize = False)

# Calculate default rate.
print(PDs_Tab)
pd_final = PDs_Tab[1] / (PDs_Tab[0] + PDs_Tab[1])
pd_final

The PD is perfectly monotonous! We should however combine the two cuts, as they contain no defaulters. We may want to tweak this result so that we obtain sufficiently representative PDs.

In [ ]:
# Adjusted cuts. Drop the second element in pbb_cuts
pbb_cuts = np.delete(pbb_cuts, 1)


After doing this, our cuts are reasonable and we can now proceed to calculate the PDs for every portfolio. For this, we need to calculate the average number of defaults for each portfolio, over the total number of cases that month.

In [ ]:
# Cut probabilities
pd_cut = pd.cut(loans['Probs'], pbb_cuts).astype('str')

# Add the PDCut variable to our Polars dataframe. Give it name 'PD_Cut'.
loans = loans.with_columns(
    pl.Series(values=pd_cut, name='PD_Cut')
    )

# Create pivot table
PD_monthly = loans.pivot(values = 'Default',
                         index = 'Portfolio',
                         on='PD_Cut',
                         aggregate_function='mean'
                         )

PD_monthly

Now we have calculated the PDs for all ratings! We ended up with exactly the Basel recommendation. On larger portfolios you will easily reach the 7-15 range.

Let's plot how our ratings look like.

In [ ]:
PD_monthly.drop("Portfolio").to_pandas().plot(subplots=True,
          layout = (3,4),
          sharex=False,
          sharey=False,
          colormap='viridis',
         fontsize=1,
         legend=False,
         linewidth=0.2);
plt.tight_layout();

We see that some months have no defaulters. With the very limited data we have that is to be expected, but in large portfolios this will be softer. In general all time series look stationary.

And that's almost it! Now we are ready to calibrate these PDs across the multiple cuts for the long-term or for IFRS 9.

## Estimating Long-Term PD Using Vasicek

To estimate the long-term PD, we need to estimate what the Z factor is given the factor and the macroeconomic factors. For this, we can use the subpackage [Time Series Analysis](https://www.statsmodels.org/stable/tsa.html#module-statsmodels.tsa) (```tsa```) of the ```statsmodel``` package. We aim to run a SARIMAX model, so we should study stationary properties, seasonality, etc.

We begin by estimating the Z factor from our data. The equation is:

$$
Z_t = \frac{\Phi^{-1}(PD_{TTC}) - \sqrt{1-\rho}\Phi^{-1}(DR_t)}{\sqrt{\rho}}
$$

where $\rho$ comes from either Basel or the asset correlation to the economy as measured by the company and $DR_t$ is the observed default rate (in our case, each element of ```PD_monthly```. The $PD_{TTC}$ is the average PD for the rating.

In [ ]:
# Defining the parameters. I will assume a fixed mortgage rho for this case (0.15). Adapt for your portfolio!
rho = 0.15

# Convert PD_monthly Polars DataFrame to Pandas DataFrame for compatibility with apply and norm.ppf
PD_monthly_pd = PD_monthly.to_pandas().fillna(0)

# Calculating the average PD per rating, from PD_monthly_pd, calculated as the average per column.
# Drop the 'Portfolio' column (if it exists as a column after to_pandas) before calculating the mean across columns (axis=0)
PD_monthly_avg = PD_monthly_pd.drop("Portfolio", axis=1).mean(axis=0)
print(PD_monthly_avg)

# Create an empty Z_t dataframe. The index should be the 'Portfolio' values.
# The columns should be the rating categories.
# Note: PD_monthly_pd will have a default integer index (0 to 107) and 'Portfolio' as a regular column.
Z_t = pd.DataFrame(index=PD_monthly_pd['Portfolio'], columns=PD_monthly_pd.columns.drop('Portfolio'))

# Populate Z_t using the Vasicek equation.
# Iterate through each rating category (column name) in Z_t
for rating_col in Z_t.columns:
    # Get the monthly default rates for the current rating category
    dr_t = PD_monthly_pd[rating_col]

    # Get the average PD for the current rating category
    pd_ttc = PD_monthly_avg[rating_col]

    # Adjust dr_t values to avoid -inf or inf from norm.ppf(0) or norm.ppf(1)
    # Replace 0 with a very small number (e.g., 1e-6) and 1 with a number slightly less than 1 (e.g., 1 - 1e-6)
    dr_t_adjusted = dr_t.apply(lambda x: 1e-6 if x == 0 else (1 - 1e-6 if x == 1 else x))

    # Calculate the numerator of the Vasicek equation
    # norm.ppf(pd_ttc) is used directly as pd_ttc values are already between (0,1)
    numerator = norm.ppf(pd_ttc) - np.sqrt(1-rho) * norm.ppf(dr_t_adjusted)

    # Calculate Z_t for the current rating column
    Z_t[rating_col] = numerator / np.sqrt(rho)

# Z_t dataframe now contains the calculated Z_t values
Z_t

Let's construct a time series from the Z factors. The first step is to turn the Portfolio index into a TimeSeries index. This way Python knows it is dealing with dates. Our data starts in January 1999 and it is monthly data. We arbitrarily set our first day as the 1st of January 1999 and add up from there.

In [ ]:
start_date = date(1999, 1, 1)

# Transform econ factors to pandas and add index
econ_factors = econ_factors.to_pandas()
econ_factors.index = [pd.to_datetime(start_date + relativedelta(months=+portfolio_month-1)) for portfolio_month in econ_factors.index]
econ_factors.drop(columns='Portfolio', inplace = True)

# Add index to Z_t
Z_t.index = [pd.to_datetime(start_date + relativedelta(months=+portfolio_month-1)) for portfolio_month in PD_monthly_pd.index]

In [ ]:
Z_t

Now we can decompose our time series to see what is happening.



In [ ]:
decomposition = seasonal_decompose(Z_t.iloc[:, 4], model='additive')
fig = decomposition.plot()
plt.show()

The time series look fairly stationary, with yearly seasonality. We are read to estimate a model!

Within the ```tsa``` package, the [SARIMAX model](https://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html#statsmodels.tsa.statespace.sarimax.SARIMAX), present in the subpackage ```statespace```, is the most general model available, allowing for Seasonality, Auto Regression, Integration, Moving Averages, and eXogenous factors. We will train a simpler ARX model (autoregressive with exogenous regressors), but you are welcome to experiment further, more complex, regressions.

Let's search for the best model for a rating, searching between 1 and 6 autoregression factors, using the macroeconomic factors as the exogenous variables.

In [ ]:
# Define the search space.
p = range(1, 6)
d = range(0, 2)
q = range(0, 2)

# Create an interative list of ps, ds, qs.
pdq = list(product(p, d, q))

# Seasonal parameters. One year back.
ps = range(0, 4)
ds = range(0, 1)
qs = range(0, 1)
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(product(ps, ds, qs))]

# Train the models for a series and test multiple values.
y = Z_t.iloc[:, 4] # Choose the fifth rating


auc_out = []

for param in pdq:
  for param_seasonal in seasonal_pdq:
      mod = SARIMAX(y,
                    exog=np.asarray(econ_factors),
                    order=param,
                    seasonal_order=param_seasonal,
                    enforce_stationarity=False,
                    enforce_invertibility=False
                    )
      results = mod.fit()
      auc_out.append([param, param_seasonal, results.aic])
      print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))

# Nicer formatting
auc_out = pd.DataFrame(auc_out,
                       columns = ['(p,q,r)', '(ps, qs, rs, S)', 'AIC'])

The warning is due to the frequency of the time series which is not provided by Pandas. You can safely ignore it.

Let's see which one is the resulting model. AIC should be minimized.

In [ ]:
auc_out.sort_values(by='AIC', ascending=True)


We can see from these values that the prefer model with parameters (5, 1, 1) 	and (3, 0, 0, 12).

Let's test the final model for these values.

In [ ]:
# ARIMA(5, 1, 1) 	(3, 0, 0, 12)
mod_BB = SARIMAX(y,
              exog=np.asarray(econ_factors),
              order=(5, 1, 1),
              seasonal_order=(3, 0, 0, 12),
              enforce_stationarity=False,
              enforce_invertibility=False)
results_BB = mod_BB.fit()

print(results_BB.summary().tables[1])

The results show what, for this dataset, the first coefficient is useful, while the second is not. On the lags and autorregressive parameters, most are not significant. We should try a simple regression of the Z_t factors on the macro only, maybe that will give better results.

It is not perfect, so we should be a bit careful with this model and possibly study other structures. I leave this as an exercise!


We can now recover our calibrated PD by inverting the process, using the estimated Z_t for any values of the economic factors we want.

- For **Basel** use the long-run estimates of the economy.
- For **IFRS 9** use the current values (or the predicted ones) for the current month.

You can calculate these PDs with the Vasicek equation.

$$
PD_{PIT} = \Phi \left(\frac{\Phi^{-1}(PD_{TTC}) - \sqrt{\rho}Z_t}{\sqrt{1-\rho}}\right)
$$

In [ ]:
# Calculate Average Economic Factors
average_econ_factors = econ_factors.mean().to_frame().T
print("Average Economic Factors:")
print(average_econ_factors)

# Predict Z-factor for Average Economic Conditions
# The model 'results_BB' was trained on Z_t.iloc[:, 4], which corresponds to the rating '(0.0, 0.222]'
predicted_z_factor = results_BB.predict(start=len(econ_factors), end=len(econ_factors), exog=average_econ_factors)
predicted_z_factor_value = predicted_z_factor.iloc[0]
print("\nPredicted Z-factor for average economic conditions (for the fifth rating):")
print(predicted_z_factor_value)

# Get the average TTC PD for the fifth rating used for training the model
pd_ttc_fifth_rating = PD_monthly_avg.iloc[4] # Corresponds to '(0.0, 0.222]'
print("\nAverage TTC PD for the fifth rating ('(0.0, 0.222]'):")
print(pd_ttc_fifth_rating)

# Calculate Calibrated PD (PD_PIT) using the inverse Vasicek formula
calibrated_pd_pit = norm.cdf( (norm.ppf(pd_ttc_fifth_rating) - np.sqrt(rho) * predicted_z_factor_value) / np.sqrt(1 - rho) )

print("\nCalibrated Point-in-Time (PIT) PD for the fifth rating under average economic conditions:")
print(calibrated_pd_pit)


We can see that the calibrated PIT PD (8.656e-07) compared to the TTC PD (0.00707) under average economic conditions is much lower. This suggests that the current average economic environment, as captured by the Z-factor, is considerably benign for this category.

And that's it! Repeating this process for every time series leads to our calibrated PDs. Now what we would do is to use the long-run estimates for the economic factors to reach a long-term PD. An alternative approach is to just run **one** regression, using the rating as a dummy predictor variable (potentially interacting with the econ factors). This is simpler, but may not have the best results. Try both approaches!

For LGD, the process is the same. The only difference is that you need to use the **downturn** economic factors instead of the long-run economic factors